In [1]:
# Package import
!pip install --user --upgrade pip
!pip install --user pyspark
!pip install --user pandas==1.3

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [1]:
# Basic Functions
import pandas as pd
import numpy as np
import random
import string

# PySpark
import pyspark
from pyspark.sql.functions import isnan, isnull
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window, SparkSession, SQLContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer,VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression

/home/ubuntu/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# Create fake dataset of random names and their age

age = list(range(10,60))
random.shuffle(age)
name =[]

for i in range(50):
    name.append(''.join(random.sample(string.ascii_lowercase, 8)))
    
df = pd.DataFrame({'Name':name,'Age':age})

In [3]:
spark =SparkSession.builder.appName('Practise').getOrCreate()
spark

In [4]:
# Create Spark dataframe and display contents
df_pyspark = spark.createDataFrame(df) 
df_pyspark.printSchema()
df_pyspark.show()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)

+--------+---+
|    Name|Age|
+--------+---+
|lbgifhdt| 57|
|ygtjswmq| 18|
|lqmxnzuy| 37|
|hyafrdte| 29|
|wiypsqxf| 50|
|zhskvjfm| 40|
|oedifgrj| 20|
|znrueqlw| 27|
|iemtualb| 59|
|sylxtova| 32|
|kncvidua| 55|
|iltknbcg| 21|
|hictpdlx| 15|
|juzwedmy| 11|
|frdvxqaj| 26|
|oymsacjw| 56|
|cjqelkvf| 17|
|gycvmsnw| 51|
|ulabpnfr| 48|
|oskjifzl| 22|
+--------+---+
only showing top 20 rows



In [104]:
# Basic Operations

In [5]:
df_pyspark.select('Name','Age').show(5)

+--------+---+
|    Name|Age|
+--------+---+
|lbgifhdt| 57|
|ygtjswmq| 18|
|lqmxnzuy| 37|
|hyafrdte| 29|
|wiypsqxf| 50|
+--------+---+
only showing top 5 rows



In [6]:
df_pyspark.dtypes

[('Name', 'string'), ('Age', 'bigint')]

In [7]:
df_pyspark.describe().show()

+-------+--------+------------------+
|summary|    Name|               Age|
+-------+--------+------------------+
|  count|      50|                50|
|   mean|    null|              34.5|
| stddev|    null|14.577379737113251|
|    min|adyfhsjk|                10|
|    max|zqwhxftd|                59|
+-------+--------+------------------+



In [8]:
# Adding Columns by manipulation

In [9]:
df_pyspark=df_pyspark.withColumn('Age after 2 years',df_pyspark['Age']+2)
df_pyspark.select('Name','Age','Age after 2 years').show(5)

+--------+---+-----------------+
|    Name|Age|Age after 2 years|
+--------+---+-----------------+
|lbgifhdt| 57|               59|
|ygtjswmq| 18|               20|
|lqmxnzuy| 37|               39|
|hyafrdte| 29|               31|
|wiypsqxf| 50|               52|
+--------+---+-----------------+
only showing top 5 rows



In [10]:
# Adding new column to pyspark dataframe
jobs = ['Data Scientist','MLops','IT','Seed Physiologist','Intern','Team Leader','Data Engineer','Lab tech','CFO','CTO','CEO']
exp_and_salary = []
for i in range(0,len(age)):
    exp_and_salary.append([name[i],random.randint(1,10),random.randint(10000,60000),random.choice(jobs)])

exp_and_salary = pd.DataFrame(exp_and_salary,columns=['name','Experience','Salary','Position'])
print(exp_and_salary.head())
exp_and_salary = spark.createDataFrame(exp_and_salary)

       name  Experience  Salary       Position
0  lbgifhdt           4   33117    Team Leader
1  ygtjswmq           3   12682  Data Engineer
2  lqmxnzuy           5   41422         Intern
3  hyafrdte           9   51908    Team Leader
4  wiypsqxf           3   15469            CFO


In [11]:
# Changing column names
exp_and_salary=exp_and_salary.withColumnRenamed('name','Name')

In [12]:
# merging another pyspark dataframe
new_df_pyspark= df_pyspark.join(exp_and_salary,["Name"])

In [13]:
new_df_pyspark= new_df_pyspark.drop('Age after 2 years')
new_df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Experience: long (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Position: string (nullable = true)



In [14]:
# Filtering

In [15]:
new_df_pyspark.filter("salary<=35000").select(['Position',"Age"]).show()

+-----------------+---+
|         Position|Age|
+-----------------+---+
|    Data Engineer| 18|
|              CTO| 40|
|              CFO| 32|
|           Intern| 59|
|Seed Physiologist| 55|
|              CFO| 56|
|              CFO| 50|
|      Team Leader| 57|
|              CTO| 20|
|Seed Physiologist| 15|
|              CEO| 11|
|      Team Leader| 41|
|              CTO| 33|
|           Intern| 51|
|              CEO| 42|
|              CEO| 10|
|         Lab tech| 22|
|      Team Leader| 53|
|               IT| 46|
|               IT| 52|
+-----------------+---+
only showing top 20 rows



In [16]:
new_df_pyspark.filter((new_df_pyspark["salary"]<=35000) & (new_df_pyspark['Age']>=15)).show(10)

+--------+---+----------+------+-----------------+
|    Name|Age|Experience|Salary|         Position|
+--------+---+----------+------+-----------------+
|adyfhsjk| 33|         6| 11752|              CTO|
|cgqjmbui| 30|         6| 33650|              CEO|
|eyczapnl| 28|         9| 20846|      Team Leader|
|fnadujlp| 42|         4| 14660|              CEO|
|gbotirew| 23|         8| 33733|              CTO|
|gslvrqbf| 16|         1| 17201|    Data Engineer|
|gycvmsnw| 51|         5| 14288|           Intern|
|hictpdlx| 15|         3| 20854|Seed Physiologist|
|iemtualb| 59|         8| 32841|           Intern|
|kncvidua| 55|         2| 14765|Seed Physiologist|
+--------+---+----------+------+-----------------+
only showing top 10 rows



In [17]:
new_df_pyspark.filter(~(new_df_pyspark["salary"]<=25000) & ~(new_df_pyspark['Age']>=15)).show(15)

+--------+---+----------+------+--------+
|    Name|Age|Experience|Salary|Position|
+--------+---+----------+------+--------+
|cakixmbs| 14|         1| 47886|Lab tech|
|juzwedmy| 11|         9| 27019|     CEO|
|uhtzkepd| 13|         7| 39197|     CTO|
+--------+---+----------+------+--------+



In [18]:
new_df_pyspark.describe().show()

+-------+--------+------------------+-----------------+------------------+-----------+
|summary|    Name|               Age|       Experience|            Salary|   Position|
+-------+--------+------------------+-----------------+------------------+-----------+
|  count|      50|                50|               50|                50|         50|
|   mean|    null|              34.5|              5.1|          33993.56|       null|
| stddev|    null|14.577379737113251|2.844615393105696|15629.639763903317|       null|
|    min|adyfhsjk|                10|                1|             11240|        CEO|
|    max|zqwhxftd|                59|               10|             59815|Team Leader|
+-------+--------+------------------+-----------------+------------------+-----------+



In [19]:
# Generating Salary Overview

In [20]:
new_df_pyspark.groupBy('Position').max().show()

+-----------------+--------+---------------+-----------+
|         Position|max(Age)|max(Experience)|max(Salary)|
+-----------------+--------+---------------+-----------+
|         Lab tech|      54|             10|      47886|
|      Team Leader|      57|             10|      51908|
|              CTO|      40|              8|      53909|
|Seed Physiologist|      55|              5|      54831|
|           Intern|      59|              8|      59112|
|            MLops|      38|              2|      49533|
|              CFO|      56|             10|      56724|
|              CEO|      47|              9|      59815|
|   Data Scientist|      44|             10|      46810|
|               IT|      58|              8|      48417|
|    Data Engineer|      49|              9|      58496|
+-----------------+--------+---------------+-----------+



In [21]:
new_df_pyspark.groupBy('Position').min().show()

+-----------------+--------+---------------+-----------+
|         Position|min(Age)|min(Experience)|min(Salary)|
+-----------------+--------+---------------+-----------+
|         Lab tech|      14|              1|      15660|
|      Team Leader|      28|              2|      20846|
|              CTO|      12|              1|      11752|
|Seed Physiologist|      15|              2|      14765|
|           Intern|      26|              2|      14288|
|            MLops|      24|              2|      38682|
|              CFO|      25|              3|      15469|
|              CEO|      10|              1|      12583|
|   Data Scientist|      27|              1|      42357|
|               IT|      39|              1|      11240|
|    Data Engineer|      16|              1|      12682|
+-----------------+--------+---------------+-----------+



In [22]:
new_df_pyspark.groupBy('Position').mean().show()

+-----------------+--------+---------------+------------------+
|         Position|avg(Age)|avg(Experience)|       avg(Salary)|
+-----------------+--------+---------------+------------------+
|         Lab tech|    34.5|           4.75|          34668.75|
|      Team Leader|    41.6|            6.8|           32339.2|
|              CTO|   26.25|           5.25|         28727.875|
|Seed Physiologist|   30.75|            3.5|          33410.75|
|           Intern|    43.6|            4.8|           40890.0|
|            MLops|    31.0|            2.0|           44107.5|
|              CFO|    38.8|            6.4|           33766.6|
|              CEO|    30.5|            5.0|33152.833333333336|
|   Data Scientist|    35.5|            5.5|           44583.5|
|               IT|   48.75|            5.0|           24307.5|
|    Data Engineer|    24.6|            5.0|           37805.8|
+-----------------+--------+---------------+------------------+



In [23]:
new_df_pyspark.groupBy('Position').count().show()

+-----------------+-----+
|         Position|count|
+-----------------+-----+
|         Lab tech|    4|
|      Team Leader|    5|
|              CTO|    8|
|Seed Physiologist|    4|
|           Intern|    5|
|            MLops|    2|
|              CFO|    5|
|              CEO|    6|
|   Data Scientist|    2|
|               IT|    4|
|    Data Engineer|    5|
+-----------------+-----+



In [ ]:
# Vishualization

In [138]:
# Converting to pandas and adding NaNs
df_new = new_df_pyspark.toPandas()
df_nulls = df_new.apply(lambda x: x.sample(frac=0.95))
df_nulls.head(10)
df_nulls.fillna('NaN Values',inplace=True)

In [ ]:
df_Pyspark = spark.createDataFrame(df_nulls)

In [25]:
#  Creating Na and replacing it
sc=spark.sparkContext
sqlContext = SQLContext(sc)

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Experience: long (nullable = true)
 |-- Salary: long (nullable = true)

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|osvfdtzb| 55|      null|  null|
|osmqklvg| 50|      null|  null|
|lpoyzwrg| 40|      null|  null|
|nfljmepz| 47|      null|  null|
|jxriuchk| 32|      null|  null|
|exdyrmth| 27|      null|  null|
|xauqdefn| 44|      null|  null|
|yvgiaruj| 58|      null|  null|
|prqwfcza| 22|      null|  null|
|adsxwjlf| 38|      null|  null|
|cbkrqytf| 34|      null|  null|
|brptfnjk| 36|      null|  null|
|onrmyksi| 15|      null|  null|
|tzekfiwr| 14|      null|  null|
|zhgfldny| 19|      null|  null|
|tuxbmgqj| 59|      null|  null|
|juerqpmc| 29|      null|  null|
|jfwvtimz| 57|      null|  null|
|lgvmbked| 24|      null|  null|
|pvtyifar| 12|      null|  null|
+--------+---+----------+------+
only showing top 20 rows



In [ ]:
# Python MLib tutorial----

In [ ]:
# MLib in Spark through DataFrame API

In [ ]:
# Simple Regression

In [ ]:
imputer = Imputer (
inputCols= ['Age','Experience','salary'],
outputCols = ["{}_imputed".format(c) for c in ['Age','Experience','salary']]).setStrategy('mean')

In [ ]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

In [ ]:
featureassembler = VectorAssembler(inputCols = ['Age','Experience'],outputCol = 'Independent_Feature')

In [ ]:
output = featureassembler.transform(df_pyspark)
output.show(5)

In [ ]:
final_pyspark = output.select("Independent_Feature","salary")
final_pyspark.show(5) 

In [ ]:
# Data Split
train_data,test_data=final_pyspark.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent_Feature',labelCol = "salary")
regressor = regressor.fit(train_data)

In [ ]:
print(regressor.coefficients)
print(regressor.intercept)

In [ ]:
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

In [ ]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

In [ ]:
# Pyspark and Sklearn

In [ ]:
import sklearn
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [ ]:
cancer_df = pd.DataFrame(data.data, columns=data.feature_names)
cancer_df['target']=data.target

In [ ]:
cancer_df_pyspark = spark.createDataFrame(cancer_df) 
cancer_df_pyspark.printSchema()
cancer_df_pyspark.show()

In [ ]:
cancer_df_pyspark.groupBy('target').count().show()
# 1 means diagnosis of breast tissues benign (B)and 0 means, the diagnosis is malignant(M)

In [ ]:
final_pyspark = output.select("Independent_Feature","salary")
final_pyspark.show(5) 

In [ ]:
diabetes_df = pd.DataFrame(data.data, columns=data.feature_names)
diabetes_df['target']=data.target

In [ ]:
diabetes_df_pyspark = spark.createDataFrame(diabetes_df) 
diabetes_df_pyspark.printSchema()
diabetes_df_pyspark.show()

In [ ]:
column_name = list(diabetes_df_pyspark.columns)
column_name.remove('target')
featureassembler = VectorAssembler(inputCols = column_name,outputCol = 'Independent_Feature')

In [ ]:
final_diabetes_pyspark = featureassembler.transform(diabetes_df_pyspark).select("Independent_Feature","target")
final_diabetes_pyspark.show(5) 

In [ ]:
train_data,test_data=final_diabetes_pyspark.randomSplit([0.65,0.35])
regressor = LinearRegression(featuresCol = 'Independent_Feature',labelCol = "target")
regressor = regressor.fit(train_data)

In [ ]:
print(regressor.coefficients)
print(regressor.intercept)
pred_results=regressor.evaluate(test_data)
pred_results.meanAbsoluteError,pred_results.meanSquaredError

In [ ]:
pred_results.predictions.show()

In [ ]:
# The dataset was generated after the hackers hacked the servers to save the company data from such activities in future 
# forensic engineers. They grabbed some features that will give us relevant metadata about the type of hackers.

# Here is a brief description of each feature:

# Session connection time: This indicates the total time session existed in minutes.
# Bytes transferred: will let us know how many megabytes were transferred during the session.
# Kali trace used: This is a kind of flag variable that indicates whether the hacker used the Kali Linux operator.
# Servers corrupted: How many servers got corrupted during the attack.
# Pages corrupted: How many pages were accessed by them illegally.
# Location: Though this meta information is also available with us, this one is of no use as hackers use VPNs
# WPM typing speed: The typing speed of those criminals based on the available logs.
# First, let’s understand what the company already knows. So, they are aware of the fact that 3 types of hackers might penetrate the attack. They are quite sure about the 2 of them, but they want us to know whether the third type of attacker was involved in this act of criminal or not.
# One key thing we should know before moving forward, i.e., forensic engineers know that hackers trade-offs, which means the number of attacks was the same for each hacker. So if there are 3 types of hackers, three of them might have equally distributed the attacks. Otherwise, the third suspect would not have been involved this time.

# Example: If all three types of attackers were the suspects, then for 100 attacks, each one will be responsible for 33.

In [ ]:
from pyspark.ml.clustering import KMeans

In [ ]:
! git clone https://github.com/farhanafayez/PySpark-K-means-Clustering-ML.git

In [ ]:
spark =SparkSession.builder.appName('Practise').getOrCreate()
dataset = spark.read.csv("/mnt/PySpark/PySpark-K-means-Clustering-ML/hack_data.csv", header=True, inferSchema=True)

In [ ]:
dataset.describe().show()

In [ ]:
dataset.dtypes

In [ ]:
# Vishualization
import matplotlib.pyplot as plt

In [ ]:
loc_group = dataset.groupBy("Location").sum("Bytes Transferred")

In [ ]:
x,y = loc_group.toPandas()["Location"].values.tolist(),loc_group.toPandas()["sum(Bytes Transferred)"].values.tolist()

In [ ]:
plt.figure(figsize=(40, 30), dpi=300)
plt.plot(x,y)
plt.show()

In [ ]:
plt.figure(figsize=(40, 30), dpi=300)
plt.bar(x,y)
plt.show()

In [ ]:
loc_group.describe().show()

In [ ]:
length((loc_group.select("Location")))

In [ ]:
feature_cols = dataset.columns
feature_cols.remove('Location')

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [ ]:
vector_assembler = VectorAssembler(inputCols =feature_cols,outputCol='features')
final_data=vector_assembler.transform(dataset)

In [ ]:
final_data

In [ ]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures',withStd=True,withMean=False)
scalerModel =scaler.fit(final_data)

In [ ]:
cluster_final_data=scalerModel.transform(final_data)

In [ ]:
cluster_final_data..select("scaledFeatures","target")

In [ ]:
predictions={}
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    model = model.transform(cluster_final_data)
    predictions[k]=model.groupBy('prediction').count()
    k_evaluator = evaluator.evaluate(model)
    print("With K={}".format(k))
    print("Error results  = " + str(k_evaluator))
    print('-'*53)

In [ ]:
for key,value in predictions.items():
    print("With K={}".format(key))
    print((value.show()))

In [ ]:
#Inference: Here, we can see that an equal number of predictions exist in the case where several clusters are only 2; 
# hence we can conclude that the third type of attacker was not involved in hacking the company’s servers.

In [ ]:
wssse_k3 = model_k3.computeCost(cluster_final_data)

In [ ]:
.select("Independent_Feature","target")

In [ ]:
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(model_k3)

In [ ]:
# Do one clustering
# do one logistic reression
# do one linear reression
# do one classification
DecisionTreeClassifier/Regressor (SPARK-19591), RandomForestClassifier/Regressor (SPARK-9478), GBTClassifier/Regressor (SPARK-9612), MulticlassClassificationEvaluator (SPARK-24101), RegressionEvaluator (SPARK-24102), BinaryClassificationEvaluator (SPARK-24103), BisectingKMeans (SPARK-30351), KMeans (SPARK-29967) and GaussianMixture (SPARK-30102).
https://spark.apache.org/docs/latest/ml-guide.html